In [ ]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt

from neuralforecast.core import NeuralForecast
from neuralforecast.models import NHITS, NBEATS

from neuralforecast.losses.numpy import mae, mse

from nixtlats import TimeGPT

%matplotlib inline

In [ ]:
with open("data/timegpt_api_key.txt", 'r') as file:
        API_KEY = file.read()

In [ ]:
df = pd.read_csv('data/daily_min_temp.csv')
df['Date'] = pd.to_datetime(df['Date'])

df.head()

In [ ]:
df['unique_id'] = 0
df = df.rename(columns={"Date": "ds", "Temp": "y"})

df = df[['unique_id', 'ds', 'y']]

df.head()

In [ ]:
fig, ax = plt.subplots(figsize=(9,6))

ax.plot(df['ds'], df['y'])
ax.set_xlabel('Time')
ax.set_ylabel('Temperature (deg C)')

fig.autofmt_xdate()
plt.tight_layout()

## Modeling 

### TimeGPT 

In [ ]:
train = df[:-365]
test = df[-365:]

print(train.shape)
print(test.shape)

### Other models 

In [ ]:
horizon = 365

models = [NHITS(h=horizon,
               input_size=2*horizon,
               max_steps=30),
         NBEATS(h=horizon,
               input_size=2*horizon,
               max_steps=30)]

In [ ]:
nf = NeuralForecast(models=models, freq='D')

In [ ]:
preds_df = nf.cross_validation(df=df, step_size=horizon, n_windows=1)

In [ ]:
preds_df.head()

## Evaluation 

In [ ]:
fig, ax = plt.subplots(figsize=(9,6))

ax.plot(preds_df['ds'], preds_df['y'], label='Actual')
ax.plot(preds_df['ds'], preds_df['NHITS'], label='NHITS', ls='--', color='red')
ax.plot(preds_df['ds'], preds_df['NBEATS'], label='NBEATS', ls=':', color='green')
ax.plot(preds_df['ds'], preds_df['TimeGPT'], label='TimeGPT', ls='-.', color='black')

ax.legend(loc='best')
ax.set_xlabel('Time')
ax.set_ylabel('Temperature (deg C)')

fig.autofmt_xdate()
plt.tight_layout()

In [ ]:
data = {'N-HiTS': [mae(preds_df['NHITS'], preds_df['y']), mse(preds_df['NHITS'], preds_df['y'])],
       'N-BEATS': [mae(preds_df['NBEATS'], preds_df['y']), mse(preds_df['NBEATS'], preds_df['y'])],
       'TimeGPT': [mae(preds_df['TimeGPT'], preds_df['y']), mse(preds_df['TimeGPT'], preds_df['y'])]}

metrics_df = pd.DataFrame(data=data)
metrics_df.index = ['mae', 'mse']

metrics_df.style.highlight_min(color='lightgreen', axis=1)